# Level 1

## What is level 1

This part is a tutorial type introduction. We assume users have read [Aviary User Interface](../user_guide/user_interface). In this doc page we discuss Level 1's interface, but also other details, such as inputs and outputs.

If you have not yet, make sure to [install Aviary](installation).

To run Aviary in the Level 1 command-line interface (CLI), type this in a terminal:

In [ ]:
!aviary

In [ ]:
# Testing Cell
import inspect

import aviary.api as av
from aviary.utils.functions import get_path
from aviary.interface.methods_for_level1 import run_level_1
from aviary.utils.doctape import check_value, glue_variable
from pathlib import Path
import pkg_resources

# obtain the default value of maximum number of iterations from function run_level_1().
max_iter = inspect.signature(run_level_1).parameters['max_iter'].default
glue_variable('max_iter', str(max_iter), md_code=False, display=True)

# check if aircraft_for_bench_GwGm.csv exists in models/test_aircraft/.
csv_file_stem = "aircraft_for_bench_GwGm"
csv_file_name = csv_file_stem + ".csv"
csv_file_path = av.get_path("models/test_aircraft/" + csv_file_name)
csv_file_path = "models/test_aircraft/" + csv_file_name

glue_variable(csv_file_stem, md_code=True, display=True)
glue_variable(csv_file_name, md_code=True, display=True)
glue_variable(csv_file_path, md_code=True, display=True)

glue_dos2unix = "dos2unix " + csv_file_name
glue_variable('dos2unix aircraft_for_bench_GwGm.csv', glue_dos2unix, md_code=False, display=True)

aviary_base_path = Path(pkg_resources.resource_filename('aviary', '.')).parent
path=get_path('models').relative_to(aviary_base_path)
glue_variable(str(path), md_code=True, display=True)
path=get_path('models/test_aircraft').relative_to(aviary_base_path)
glue_variable(str(path), md_code=True, display=True)

check_value(av.EquationsOfMotion.HEIGHT_ENERGY.value, 'height_energy')
check_value(av.EquationsOfMotion.TWO_DEGREES_OF_FREEDOM.value, '2DOF')
check_value(av.EquationsOfMotion.SOLVED_2DOF.value, 'solved_2DOF')


We'll now discuss the different parts of the CLI, starting with how to run an Aviary problem.

## First level 1 run

We are ready to run some models. By default, Aviary runs the optimizer for up to {glue:md}`max_iter` iterations. In order to reduce the run time, we will limit the maximum number of iterations to 1. It will be faster for these examples, but you will not get optimal solutions. Issue the following command:

In [ ]:
!aviary run_mission models/test_aircraft/aircraft_for_bench_GwGm.csv --optimizer IPOPT --max_iter 1

The argument {glue:md}`models/test_aircraft/aircraft_for_bench_GwGm.csv` shows where the aircraft csv file lives. This argument must be one of the following three options:

- an absolute path,
- a relative path relative to the current working directory,
- a relative path relative to the Aviary package.

Aviary searches for the given dataset in this order. If a dataset file exists in multiple directories, the first one Aviary finds will be used.

```{note}
When you run Aviary all of the outputs will be saved in the location where you run the command or run script.
Outputs files such as optimization history and Dymos output files are saved at that location.
A folder named 'report' will also be created there, and contains all of the associated report files for your runs.
These reports include [optimization and trajectory reports generated by OpenMDAO and Dymos](https://openmdao.org/newdocs/versions/latest/features/reports/reports_system.html).
```

## Warning messages

During your first run you may notice some warning messages in the Aviary output. Frequently seen warnings are:

- **PromotionWarning:** Issued when there is ambiguity due to variable promotion (an [OpenMDAO warning](https://openmdao.org/newdocs/versions/latest/features/warning_control/warnings.html)).
- **RuntimeWarning:** Issued for warnings about dubious runtime features (a [Python warning](https://docs.python.org/3/library/warnings.html)).
- **UserWarning:** Issued for warnings about potential OpenMDAO, Dymos, and/or Aviary problems.
- **DerivativesWarning:** Issued when the approximated partials or coloring cannot be evaluated as expected (an [OpenMDAO warning](https://openmdao.org/newdocs/versions/latest/features/warning_control/warnings.html)).

Some of these warnings are expected and can be ignored. For example, the following warnings are expected when running the {glue:md}`aircraft_for_bench_GwGm.csv` model:

```
~/workspace/OpenMDAO/openmdao/core/group.py:326: PromotionWarning:<class Group>: Setting input defaults for input 'mission:takeoff:ascent_duration' which override previously set defaults for ['auto', 'prom', 'src_shape', 'val'].
~/workspace/dymos/dymos/phase/phase.py:2007: RuntimeWarning: Invalid options for non-optimal control 'throttle' in phase 'groundroll': lower, upper
  warnings.warn(f"Invalid options for non-optimal control '{name}' in phase "
~/workspace/dymos/dymos/phase/phase.py:1967: UserWarning: Phase time options have no effect because fix_initial=True or input_initial=True for phase 'traj.phases.rotation': initial_ref
  warnings.warn(f'Phase time options have no effect because fix_initial=True '
```

For now, we can ignore the warning messages and continue.
<!-- TODO: expand this section -->

## Level 1 run options
In addition to a model input file, this option has additional options that are important, as seen here:

In [ ]:
!aviary run_mission

In [ ]:
# Testing Cell
from aviary.interface.methods_for_level1 import _setup_level1_parser
from aviary.utils.doctape import glue_variable
import inspect

# glue all the options of "aviary run_mission"
source_code = inspect.getsource(_setup_level1_parser)
new_line = ''
for ch in source_code:
    if ch == '\n':
        new_line = new_line.strip()
        if new_line.startswith("\"-") or new_line.startswith("'-"):
            skey = new_line.split(",")[0][1:-1]
            if skey == "-o":
                skey = "-o --outdir"
            glue_variable(skey, md_code=True, display=True)
        new_line = ''
    else:
        new_line += ch


Let us discuss these in more detail:

- {glue:md}`-o --outdir`: Use specified directory to write output. The default is the current directory.

- {glue:md}`--optimizer`: Name of optimizer. Choices are: `SNOPT`, `IPOPT`, `SLSQP`, and `None`. The default is `SNOPT`. If optimizer is `None`, it will be set to `IPOPT` or `SNOPT` depending on the analysis scheme. 
- {glue:md}`--phase_info`: Path to phase info file. If not provided, it is `default_phase_info/two_dof.py` if Mission origin is `TWO_DEGREES_OF_FREEDOM` and `default_phase_info/height_energy.py` for `simple`.

- {glue:md}`--max_iter`: Maximum number of iterations. Default is {glue:md}`50`.

- {glue:md}`--shooting`: Use shooting instead of collocation.

For the  {glue:md}`aircraft_for_bench_GwGm.csv` example so far, we have used the `IPOPT` optimizer because it is publicly available through [pyOptSparse](https://mdolab-pyoptsparse.readthedocs-hosted.com/en/latest/).

In [ ]:
# Testing Cell
from aviary.interface.cmd_entry_points import _command_map
run_mission = 'run_mission'
_command_map[run_mission];
command = "aviary " + run_mission
glue_variable(run_mission, md_code=True, display=True)
glue_variable(command, md_code=True, display=True)
f_to_a = 'fortran_to_aviary'
_command_map[f_to_a];
glue_variable(f_to_a, md_code=True, display=True)

## Aviary run_mission command

The Level 1 CLI (i.e. {glue:md}`run_mission` option) is designed to offer the lowest barrier to entry for new users.
Analysts who have experience with legacy tools, such as `FLOPS` and `GASP`, should find the switch from FORTRAN namelists to csv-based input decks to be straightforward. Aviary input decks allow the user the ability to set aircraft characteristics and basic mission parameters,
such as cruise Mach number and altitude, in a simple text-based format that does not require any familiarity with Python or OpenMDAO.

Aviary can then be executed by calling {glue:md}`aviary run_mission` with the csv input deck.
Although the order of the variables in the input deck is not important, you might find it helpful to separate the variables based on if they're used as initial guesses or in the metadata.

## First aircraft model

We have a few sample aircraft csv files in {glue:md}`aviary/models`. They are all `.csv` files. For example, {glue:md}`aircraft_for_bench_GwGm.csv` (in {glue:md}`aviary/models/test_aircraft`) is a large single aisle aircraft mission model.

In [ ]:
# Testing Cell
from aviary.utils.functions import get_model, get_path
from aviary.utils.doctape import glue_variable
get_path('models/test_aircraft/aircraft_for_bench_GwGm.csv')

csv_snippet = '```\n'
filename = 'aircraft_for_bench_GwGm.csv'
with open(get_model(filename)) as f_in:
    lines = f_in.readlines()
    n = len(lines)
    l_aircraft = []
    l_mission = []
    l_settings = []
    l_init = []
    l_GASP = []
    cnt_aircraft = 0
    cnt_mission = 0
    cnt_settings = 0
    cnt_initial = 0
    cnt_gasp = 0
    for idx in range(n):
        s = lines[idx]
        if "aircraft:wing:aspect_ratio" in s:
            glue_variable("aircraft:wing:aspect_ratio", md_code=True, display=True)
            glue_variable("Aircraft.Wing.ASPECT_RATIO", md_code=True, display=True)
            pass
        elif "climb_range" in s:
            glue_variable("climb_range", md_code=True, display=True)
        if s.startswith("aircraft") and cnt_aircraft < 3:
            l_aircraft.append(s)
            cnt_aircraft += 1
        elif s.startswith("mission") and cnt_mission < 3:
            l_mission.append(s)
            cnt_mission += 1
        elif s.startswith("settings") and cnt_settings < 3:
            l_settings.append(s)
            cnt_settings += 1
        elif s.count(",") == 1 and cnt_initial < 3:
            l_init.append(s)
            cnt_initial += 1
        elif s.startswith("INGASP") and cnt_gasp < 3:
            l_GASP.append(s)
            cnt_gasp = cnt_gasp + 1
    l = l_aircraft
    l.extend(["......\n"])
    l.extend(l_mission)
    l.extend(["......\n"])
    l.extend(l_settings)
    l.extend(["......\n"])
    l.extend(l_init)
    l.extend(["......\n"])
    l.extend(l_GASP)
    l.extend(["......\n"])
    csv_snippet+=''.join(l)

csv_snippet+='...\n```'
glue_variable('csv_snippet', csv_snippet, md_code=False, display=True)

Open {glue:md}`aircraft_for_bench_GwGm.csv` using your favorite editor (e.g., MS Excel or VS Code). If you choose to use MS Excel, you need to take extra steps to make sure that it is in the original format. We recommend running dos2unix first (e.g. {glue:md}`dos2unix aircraft_for_bench_GwGm.csv`).

The file {glue:md}`aircraft_for_bench_GwGm.csv` is a data input file modeling large single aisle aircraft which was converted from a `GASP` input deck.
Here is a snippet of the file:

```{glue:md} csv_snippet
:format: myst
```

The input `.csv` file defines the aircraft and mission. In an Aviary input file you see a list of Aviary variables, their values, and units. Aviary variables are colon delimited strings (e.g. `aircraft:wing:aspect_ratio`). An Aviary variable name usually has three words. The first word is `aircraft` or `mission`, the second word is a subsystem name and the third is the variable name. Each variable has a value and units. Aviary requires units for every variable because it reduces errors and is a good engineering practice.

```{note}
If you have used the {glue:md}`fortran_to_aviary` tool to create a `.csv` file you must check the outputted variable values and units to ensure the file is valid and what you expect.
```

Be aware that some variables do not follow the standard naming format (e.g. {glue:md}`climb_range`). These are used for the initial guessing of the trajectory. They are intentionally separate to prevent conflicts with some similarly named variables. They are only used once to set up the problem and then are discarded.

Finally we see a list of `GASP` variables that are not converted. Not all variables are converted to Aviary right now. They may represent some features in `FLOPS` and `GASP` that we haven't implemented in Aviary yet.

To find information about a variable (e.g. description, data type, etc.), users should read the [Variable Metadata Doc](../user_guide/variable_metadata).

There are special rules for the mapping from the input file variable names to the metadata. For example, variable `aircraft:wing:aspect_ratio` in {glue:md}`aircraft_for_bench_GwGm.csv` is mapped to{glue:md}`Aircraft.Wing.ASPECT_RATIO` in `aviary/variable_info/variable_meta_data.py`. So, the first part (e.g., `aircraft` or `mission`) is mapped to the same word but with the first letter capitalized (e.g., `Aircraft` or `Mission`). The third word is all caps (e.g., `ASPECT_RATIO`). The middle part (e.g., `wing`) is a little more complicated. In most cases, this part capitalizes the first letter (e.g., `Wing`). The following words have special mappings:

- `air_conditioning -> AirConditioning`
- `anti_icing -> AntiIcing`
- `blended_wing_body -> BWB`
- `crew_and_payload -> CrewPayload`
- `horizontal_tail -> HorizontalTail`
- `landing_gear -> LandingGear`
- `tail_boom -> TailBoom`
- `vertical_tail -> VerticalTail`

This can be summarized as to capitalize the leading letter and to capitalize the first letter after special character “`_`”.

File {glue:md}`aviary/variable_info/variables.py` is a variable hierarchy that is for a single mission. Each mission gets a copy of this hierarchy. Below is a snippet of this file:

In [ ]:
import aviary.api as av

# Get the path of variables.py within the aviary package
file_path = av.get_path('variable_info/variables.py')
glue_variable('aviary/variable_info/variables.py', md_code=True, display=True)

# Read and print the first 15 lines of the file
with open(file_path, 'r') as file:
    for i in range(18):
        print(file.readline().strip('\n'))

Aviary variables are always set to default values before the input file is read in. Then Aviary will update the values based on the user-provided `.csv` input file. If you want to set different values, you can set them in the `.csv` input file.

In [ ]:
# Testing Cell
import aviary.api as av
from aviary.utils.doctape import glue_variable

# check if the following model files exist.

other_models = '```\n'
filename = 'models/N3CC/N3CC_generic_low_speed_polars_FLOPSinp.csv'
filepath = av.get_path(filename)
# glue_variable('aviary/' + filename, md_code=True, display=True)
other_models += 'aviary/' + filename + '\n'

filename = 'models/large_single_aisle_1/large_single_aisle_1_GwGm.csv'
filepath = av.get_path(filename)
# glue_variable('aviary/' + filename, md_code=True, display=True)
other_models += 'aviary/' + filename + '\n'
glue_variable('large_single_aisle_1', md_code=True)

filename = 'models/small_single_aisle/small_single_aisle_GwGm.csv'
filepath = av.get_path(filename)
# glue_variable('aviary/' + filename, md_code=True, display=True)
other_models += 'aviary/' + filename + '\n'

filename = 'models/test_aircraft/converter_configuration_test_data_GwGm.csv'
filepath = av.get_path(filename)
# glue_variable('aviary/' + filename, md_code=True, display=True)
other_models += 'aviary/' + filename

glue_variable('other_models', other_models, md_code=False, display=True)

## Other available input files

Other input files reside in aviary/models. They include:

```{glue:md} other_models
:format: myst
```

For example, to run {glue:md}`large_single_aisle_1` model, you execute the following command:

In [ ]:
!aviary run_mission models/large_single_aisle_1/large_single_aisle_1_GwGm.csv --max_iter 1 --optimizer IPOPT

For FLOPS-derived models, let us run:

In [ ]:
!aviary run_mission models/test_aircraft/aircraft_for_bench_FwFm.csv --max_iter 1 --optimizer IPOPT

## Aviary fortran_to_aviary command

```{note}
You only need to use the `fortran_to_aviary` command if you are converting a FLOPS or GASP deck to Aviary's csv format.
```

This {glue:md}`fortran_to_aviary` tool (see [Aviary Commands](../user_guide/aviary_commands) for details) converts FORTRAN namelists into Aviary's csv based format using the mappings found in the `historical_name` section of the [variable_meta_data](../user_guide/variable_metadata). The resulting csv is automatically sorted into three sections:
1. **Input Values:**
    Any FORTRAN variables that were mapped to input variables in Aviary components
2. **Initial Guesses:**
    Some variables are only used as initial guesses for the trajectory.
    These are displayed separately from the Input Values because they will not be passed directly to components
3. **Unconverted Values:**
    If the {glue:md}`fortran_to_aviary` converter can't find an Aviary variable that matches the FORTRAN variable, it is added to the end of the csv file.
    We recommend that you check this section after converting a namelist to ensure that there aren't any variables you expected to be converted here.
    Many of these unconverted variables represent features or options that are not used in Aviary and can be safely ignored. Variables related to mission definition are important, but Aviary defines mission profiles in a significantly different way. Currently, the user must build a new [mission definition file](../examples/simple_mission_example) that recreates the mission.
    Aviary will ignore unconverted variables when loading the csv, so you can safely leave them.


In [ ]:
# Testing Cell
from aviary.interface.cmd_entry_points import _command_map
_command_map['fortran_to_aviary'];

## Phase info

Aviary runs depend not only on input `*.csv` files, but also on `phase_info` dictionaries. A `phase_info` dictionary defines the mission and any analysis settings used within Aviary. It is used by Aviary to build a mission trajectory (e.g., from take-off to landing).

A [Python dictionary](https://www.w3schools.com/python/python_dictionaries.asp) is a set of key-value pairs. Most keys in `phase_info` are self-explained. Users should read Dymos' [Phase Options](https://openmdao.github.io/dymos/api/phase_api.html). For more details about phase_info keys, especially their example usages, please read [onboarding phase information](input_csv_phase_info).

### Default TWO_DEGREES_OF_FREEDOM phases

The {glue:md}`aviary/interface/default_phase_info/two_dof.py` file is shown below:

In [ ]:
import aviary.api as av

# Get the path of variables.py within the aviary package
file_path = av.get_path('interface/default_phase_info/two_dof.py')

# Read the file contents
with open(file_path, 'r') as file:
    # Find the line that starts with 'phase_info ='
    for line in file:
        if line.startswith('phase_info ='):
            print(line.strip('\n'))
            break
    
    # Print the remaining lines
    for line in file:
        print(line.strip('\n'))

file_path = "aviary/interface/default_phase_info/two_dof.py"
glue_variable(file_path)

In [ ]:
# Testing Cell
from aviary.interface.default_phase_info.two_dof import phase_info
from aviary.interface.utils.check_phase_info import check_phase_info, TWO_DEGREES_OF_FREEDOM
from aviary.utils.doctape import check_value, glue_keys

check_phase_info(phase_info, TWO_DEGREES_OF_FREEDOM)
# glue_variable("((1.0, 100.0), 's')", md_code=True)
duration_bounds_val, duration_bounds_unit = phase_info['groundroll']['user_options']['duration_bounds']
duration_bounds_lower = duration_bounds_val[0]
duration_bounds_upper = duration_bounds_val[1]
check_value(str(duration_bounds_unit), "s")
check_value(duration_bounds_lower, 1.0)
check_value(duration_bounds_upper, 100.0)

duration_ref_val, duration_ref_unit = phase_info['groundroll']['user_options']['duration_ref']
check_value(duration_ref_val, 50.0)
check_value(duration_ref_unit, "s")
velocity_lower_val, velocity_lower_unit = phase_info['groundroll']['user_options']['velocity_lower']
check_value(velocity_lower_val, 0)
check_value(velocity_lower_unit, 'kn')
velocity_upper_val, velocity_upper_unit = phase_info['groundroll']['user_options']['velocity_upper']
check_value(velocity_upper_val, 1000)
check_value(velocity_upper_unit, 'kn')
velocity_ref_val, velocity_ref_unit = phase_info['groundroll']['user_options']['velocity_ref']
check_value(velocity_ref_val, 150)
check_value(velocity_ref_unit, 'kn')

distance_ref_val, distance_ref_unit = phase_info['groundroll']['user_options']['distance_ref']
check_value(distance_ref_val, 3000)
check_value(distance_ref_unit, 'ft')
distance_lower_val, distance_lower_unit = phase_info['groundroll']['user_options']['distance_lower']
check_value(distance_lower_val, 0)
check_value(distance_lower_unit, 'ft')
distance_upper_val, distance_upper_unit = phase_info['groundroll']['user_options']['distance_upper']
check_value(distance_upper_val, 10.e3)
check_value(distance_upper_unit, 'ft')

mass_ref_val, mass_ref_unit = phase_info['groundroll']['user_options']['mass_ref']
check_value(mass_ref_val, 150_000)
check_value(mass_ref_unit, 'lbm')
mass_lower_val, mass_lower_unit = phase_info['groundroll']['user_options']['mass_lower']
check_value(mass_lower_val, 0)
check_value(mass_lower_unit, 'lbm')
mass_upper_val, mass_upper_unit = phase_info['groundroll']['user_options']['mass_upper']
check_value(mass_upper_val, None)
check_value(mass_upper_unit, 'lbm')

throttle_val, throttle_unit = phase_info['groundroll']['initial_guesses']['throttle']
check_value(throttle_val, [0.956, 0.956])
check_value(throttle_unit, 'unitless')

num_segments = phase_info['groundroll']['user_options']['num_segments']
check_value(num_segments, 1)

order = phase_info['groundroll']['user_options']['order']
check_value(order, 3)

connect_initial_mass = phase_info['groundroll']['user_options']['connect_initial_mass']
check_value(connect_initial_mass, False)

fix_initial = phase_info['groundroll']['user_options']['fix_initial']
check_value(fix_initial, True)

glue_keys(phase_info, display=True)


The file `default_phase_info/two_dof.py` contains the following phases:

{glue:md}`groundroll`, {glue:md}`rotation`, {glue:md}`ascent`, {glue:md}`accel`, {glue:md}`climb1`, {glue:md}`climb2`, {glue:md}`cruise`, {glue:md}`desc1`, {glue:md}`desc2`.

All of the above phases belong to mission. No pre-mission phase is provided. If `pre_mission` is missing, a default `pre_mission` is provided:
```
    'pre_mission': {
        'include_takeoff': True,
        'external_subsystems': [],
    }
```

Similarly, if no post-mission phase is provided, then a default `post_mission` is provided:
```
    'post_mission': {
        'include_landing': True,
        'external_subsystems': [],
    },
```

For `GASP' missions, taxi is considered part of pre-mission and landing is considered part of post-mission.

#### Groundroll phase

Let us discuss the groundroll phase in detail as an example.

In {glue:md}`groundroll` phase, we are given {glue:md}`duration_bounds` of range `((1., 100.), 's')` and a reference value {glue:md}`duration_ref` of value `(50., 's')`. Here, {glue:md}`duration_bounds` is a tuple of (lower, upper) bounds for the duration of the integration variable across the phase and {glue:md}`duration_ref` is the unit-reference value for the duration of the integration variable across the phase (see [Dymos Variables](https://openmdao.github.io/dymos/features/phases/variables.html)). This implies a time range of `(0.2, 2)` in {glue:md}`groundroll` phase, or:
```
0.02 ≤ traj.groundroll.t_duration ≤ 2.
```
We see {glue:md}`velocity_lower` of value `(0, 'kn')`, {glue:md}`velocity_upper` of value `(1000, 'kn')`, and {glue:md}`velocity_ref` of value `(150, 'kn')`. They result in `velocity` as a design variable with a range: `(0, 6.66667)`, or
```
0 ≤ traj.groundroll.states:velocity ≤ 6.66667.
```
Similarly, we get:
```
0 ≤ traj.groundroll.states:distance ≤ 3.33333
```
The mass parameter is a little bit different because we have {glue:md}`mass_upper`: `(None, 'lbm')` (and {glue:md}`mass_lower`: `(0, 'lbm')`). In this situation, OpenMDAO replaces `None` with an actual upper bound that is very large. We get:
```
0 ≤ traj.groundroll.states:mass ≤ 6.66667e+15.
```
Comparing to `velocity` and `distance`, `mass` is not scaled to the same range. This is okay because aircraft mass is actually a constant in this phase. A huge upper bound will not have an impact.

In {glue:md}`groundroll` phase, initial guess of {glue:md}`throttle` setting is set to maximum `(0.956, 0.956)`. Aviary sets a phase parameter:
```
Dynamic.Mission.THROTTLE = 0.956.
```
For the [`COLLOCATION`](https://openmdao.github.io/dymos/getting_started/collocation.html) setting, there is one [segment](https://openmdao.github.io/dymos/getting_started/intro_to_dymos/intro_segments.html) (`'num_segments': 1`) and polynomial interpolation degree is 3 (`'order': 3`). Increasing the number of segments and/or increasing the degree of polynomial will improve accuracy but will also increase the complexity of computation. For groundroll, it is unnecessary.

Since {glue:md}`groundroll` is the first phase, {glue:md}`connect_initial_mass` is set to `False`. {glue:md}`fix_initial` is set to `True`, but {glue:md}`fix_initial_mass` to `False` because aircraft total mass is a design parameter.

For parameter {glue:md}`mass_defect_ref`, we see values to `(150_000, 'lbm')`. Defect is a residual that measures how accurately the proposed state and control history obeyed the ODE governing the system dynamics. For distance, the defect reference {glue:md}`distance_ref` is `3000` ft.

#### Other phases

We will not discuss the other phases in detail.

Phase {glue:md}`climb1` is for climb up to 10,000 ft and phase {glue:md}`climb2` is for climb to cruise phase. Phase {glue:md}`desc1` is for descent down to `10,000` ft and phase {glue:md}`desc2` is for descent from `10,000` ft down to `1,000` ft.

In [ ]:
# Testing Cell
from aviary.interface.default_phase_info.height_energy import phase_info
from aviary.interface.utils.check_phase_info import check_phase_info, HEIGHT_ENERGY
from aviary.utils.doctape import check_value, get_all_keys

# Get the path of variables.py within the aviary package
file_path = av.get_path('interface/default_phase_info/height_energy.py')
# Now the file exists. Glue it.
file_path = 'aviary/interface/default_phase_info/height_energy.py'
glue_variable('aviary/interface/default_phase_info/height_energy.py', file_path, md_code=True, display=True)

# glue new phase name
for key in ['pre_mission', 'climb', 'descent', 'post_mission']:
    glue_variable(key, md_code=True, display=True)

### Default HEIGHT_ENERGY phases

The file {glue:md}`aviary/interface/default_phase_info/height_energy.py` contains the following phases:

{glue:md}`pre_mission`, {glue:md}`climb`, {glue:md}`cruise`, {glue:md}`descent`, {glue:md}`post_mission`

The differences between `HEIGHT_ENERGY` and `TWO_DEGREES_OF_FREEDOM` phases are due to how `FLOPS` and `GASP` implement trajectory analysis.

```{note}
File `aviary/interface/default_phase_info/height_energy.py` has a {glue:md}`pre_mission` phase and a {glue:md}`post_mission` phase. In {glue:md}`pre_mission`, `takeoff` is the simplified takeoff and in `post_mission`, `landing` is the simplified landing. For `HEIGHT_ENERGY` missions, there are [detailed takeoff and landing](../user_guide/FLOPS_based_detailed_takeoff_and_landing) available. But they are not used in `default_phase_info/flops.py`. The other phases are mission phases.
```

## Outputted files and how to read them

```{note}
Please also see [this doc page](../user_guide/postprocessing_and_visualizing_results) for more info about the Aviary dashboard and the information is contains.
We usually expect that users access these reports through the dashboard, though you can also access the html files directly.
```

Each standard run generates several output files. Which output files are generated depend on the run options. In this section, we assume that we've set `max_iter` = {glue:md}`max_iter` to ensure convergence. The following screenshots used in this article are all from a run using {glue:md}`aircraft_for_bench_GwGm.csv`.

First, there is always a sub-folder `reports/case_name` where `case_name` is the csv file name. In our case, it is `reports/aircraft_for_bench_GwGm`. It contains a few HTML files:

- `driver_scaling_report.html`
- `inputs.html`
- `n2.html`
- `opt_report.html`
- `total_coloring.html`
- `traj_linkage_report.html`
- `traj_results_report.html`

### File `driver_scaling_report.html`

This file is a summary of driver scaling information. After all design variables, objectives, and constraints are declared and the problem has been set up, this report presents all the design variables and constraints in all phases as well as the objectives. The file is divided to three blocks: `Design Variables`, `Constraints`, and `Objectives`. It contains the following columns: name, size, indices, driver value and units, model value and units, ref, ref0, scaler, adder, etc. It also shows `Jacobian Info` - responses with respect to design variables (`DV`). A screen shot of design variables is shown here:

> ![Sample driver_scaling_report.html (top lines)](images/driver_scaling_report_top.PNG)

This file is needed when you are debugging. New users can skip it.

### File `inputs.html`

File `inputs.html`  is a sortable and filterable input report of input variables in different phases. It contains all the input variables (with possibly duplicate names for different phases) but only shows those in `Source is IVC` (abbreviation for [IndepVarComp](https://openmdao.org/newdocs/versions/latest/features/core_features/working_with_components/indepvarcomp.html)) on opening up. Users can choose to show other inputs by selecting and deselecting this checkbox. Users can filter their inputs by input name, source name, units, shape, tags,  values (`Val`), or design variables (`Source is DV`). Here is a screen shot (top part) when it is opened:

> ![Sample inputs.html (top lines)](images/inputs_html_top.PNG)

New users can choose to use `input_list.txt` instead (see below) if `settings:verbosity` is set to 2 or higher. Note that this file lives in the current folder.

### File `n2.html`

[N2](https://openmdao.org/newdocs/versions/latest/features/model_visualization/n2_details/n2_details.html), sometimes referred to as an eXtended Design Structure Matrix (XDSM), is a powerful tool for understanding your model in OpenMDAO. It is an N-squared diagram in the shape of a matrix representing functional or physical interfaces between system elements. It can be used to systematically identify, define, tabulate, design, and analyze functional and physical interfaces.

```{note}
We *strongly* recommend that you understand N2 diagrams well, especially if you are debugging a model or adding external subsystems. Here is a [doc page](https://openmdao.github.io/PracticalMDO/Notebooks/ModelConstruction/using_n2.html) featuring an informative video on how to use N2 diagrams effectively. MDO lab has another resource for learning about N2 and [XDSM](https://mdolab-pyxdsm.readthedocs-hosted.com/).
```

 Here is a screenshot of an N2 diagram after we run `aviary run_mission models/test_aircraft/aircraft_for_bench_GwGm.csv`.

> ![Sample n2.html (on start)](images/N2_start.PNG)

At level 1, we are interested in the phases, so let us `Hide solvers`. So from `model`, zoom into `traj`, then into `phases`. We see the phases from `groundroll` to `desc2` as expected. To see how those phases are connected, we click on `Show all connections in view`. Here is what we see:

> ![Sample n2.html (on phases)](images/N2_phases.PNG)

The solid arrow connections show how outputs of one phase feed as inputs to the next. As you can see, the input of each phase is linked from the previous phase and its output is linked to the next phase. This is pretty much what we expect. The dashed arrows are links to and/or from other places not in the view. If you are curious where those links go, you must zoom out. This can be done by clicking the vertical bar on the left. Let us click it twice. Here is what we get:

> ![Sample n2.html (on phases)](images/N2_phases_links.PNG)

For `TWO_DEGREES_OF_FREEDOM` missions, there is a takeoff subsystem (including `taxi` phase) within `pre_mission`. For `HEIGHT_ENERGY` missions, there is a takeoff subsystem within `pre_mission` in which a `takeoff` phase is added. It is added after `static_analysis` in `add_pre_mission_systems` method.

Similarly, there is a `landing` phase within `post_mission`.

### File `opt_report.html`

This file is OpenMDAO Optimization Report. All values are in unscaled, physical units. On the top is a summary of the optimization, followed by the objective, design variables, constraints, and optimizer settings. Here is a screenshot:

> ![Sample opt_report.html (top lines)](images/opt_report_top.PNG)

This file is important when dissecting optimal results produced by Aviary.

### Coloring files

There is a sub-folder `coloring_files`. Those are used internally by OpenMDAO for derivative computation. Users should generally skip those files.

### File `traj_linkage_report.html`

This is a dymos linkage report in a customized N2 diagram. It provides a report detailing how phases are linked together via constraint or connection. It can be used to identify errant linkages between fixed quantities.

> ![Sample traj_linkage_report.html (top of timeseries)](images/traj_linkage_report_top.PNG)

We clearly see how those mission phases are linked.

### File `traj_results_report.html`

```{note}
This is one of the most important files produced by Aviary. It will help you visualize and understand the optimal trajectory produced by Aviary.
```

This file contains timeseries and phase parameters in different tabs. For our {glue:md}`aircraft_for_bench_GwGm` run, they are: {glue:md}`groundroll`, {glue:md}`rotation`,  {glue:md}`ascent`, {glue:md}`accel`, {glue:md}`climb1`, {glue:md}`climb2`, {glue:md}`cruise`, {glue:md}`desc1`, and {glue:md}`desc2` parameters. On the timeseries tab, users can select which phases to view. The following are the top of timeseries tab and ascent parameters tab:

> ![Sample traj_results_report.html (top part of timeseries)](images/traj_results_report_top.PNG)

Let's find the altitude chart. Move the cursor to the top of climb2. We see that the aircraft climbs to 37500 feet at 1200 second. Then it enters to cruise phase. At 28090 second, it starts descent from 40000 feet.

> ![Sample traj_results_report.html (altitude)](images/traj_results_report_altitude.PNG)

Let's switch to `ascent` tab. We see the following:

> ![Sample traj_results_report.html (top lines) of ascent](images/traj_results_report_top_ascent.PNG)

This file is quite important. Users should play with it and try to grasp all possible features. For example, you can hover the mouse over the solution points to see solution value; you can save the interesting images; you can zoom into a particular region for details, etc.

### Optimizer output

If `IPOPT` is the optimizer, `IPOPT.out` is generated. If `SLSQP` is the optimizer and `pyOptSparseDriver` is the driver, `SLSQP.out` is generated. Generally speaking, `IPOPT` and `SNOPT` converge Aviary optimization problems better than `SLSQP`, but `SLSQP` is bundled with Scipy by default, making it more widely available.

### SQLite database file

There is a `.db` file after run. By default, it is `problem_history.db` in the current directory. This is an SQLite database file. In level 2 and level 3, we will be able to choose a different name. Our run is recorded into this file. You generally shouldn't need to parse through this file on your own, but it is available if you're seeking additional problem information.

In [ ]:
# Testing Cell
import aviary.api as av
from aviary.utils.doctape import get_attribute_name, glue_variable
Verbosity = av.Verbosity;

verbosity = get_attribute_name(av.Settings,av.Settings.VERBOSITY)
glue_variable('VERBOSITY',verbosity, md_code=True)
glue_variable('BRIEF',av.Verbosity.BRIEF.name, md_code=True)
glue_variable('VERBOSE',av.Verbosity.VERBOSE.name, md_code=True)
# glue_variable(av.Settings.VERBOSITY)

### Plain text inputs and outputs

If {glue:md}`VERBOSITY` is set to `2` ({glue:md}`VERBOSE`) or higher, `input_list.txt` and `output_list.txt` are generated. We recommend this parameter be set to `1` ({glue:md}`BRIEF`) for beginners. See [Coding Standards](../developer_guide/coding_standards.ipynb) for more details.

Let us look at `input_list.txt`:

> ![Sample input_list.txt (top lines)](images/input_list_top.PNG)

In this screenshot, we see a tree structure. There are three columns. The left column is a list of variable names. The middle column is the value and the right column is the [promoted](https://openmdao.org/newdocs/versions/latest/basic_user_guide/multidisciplinary_optimization/linking_vars.html) variable name. `pre_mission` is a phase, `static_analysis` is a subgroup which contains other subcomponents (e.g. `propulsion_group`). `engine` is a component under which is a list of input variables (e.g. `aircraft:engine:scaled_sls_thrust`).

An input variable can appear under different phases and within different components. Note that its values can be different because its value has been updated during the computation. On the top-left corner is the total number of inputs. That number counts the duplicates because one variable can appear in different phases. Aviary variable structure are discussed in [Understanding the Variable Hierarchy](https://github.com/OpenMDAO/Aviary/blob/main/aviary/docs/user_guide/variable_hierarchy) and [Understanding the Variable Metadata](https://github.com/OpenMDAO/Aviary/blob/main/aviary/docs/user_guide/variable_metadata).

If you zoom the N2 diagram into `propulsion_group` and `gasp_based_geom`, you see the exact same tree structure:

> ![Sample N2 Diagram (top blocks)](images/N2_top.PNG)

File `output_list.txt` follows the same pattern. But there is another tree for implicit outputs. That helps when debugging is needed.

### Additional messages

When Aviary is run, some messages are printed on the command line and they are important. For example, the following constraint report tells us whether our desired constraints are met:

```
--- Constraint Report [traj] ---
    --- groundroll ---
        None
    --- rotation ---
        [final]   0.0000e+00 == normal_force [lbf]
    --- ascent ---
        [final]   5.0000e+02 == altitude [ft]
        [path]    0.0000e+00 <= load_factor <= 1.1000e+00  [unitless]
        [path]    0.0000e+00 <= fuselage_pitch <= 1.5000e+01  [deg]
    --- accel ---
        [final]   2.5000e+02 == EAS [kn]
    --- climb1 ---
        [final]   1.0000e+04 == altitude [ft]
    --- climb2 ---
        [final]   3.7500e+04 == altitude [ft]
        [final]   1.0000e-01 <= altitude_rate  [ft/min]
        [final]   8.0000e-01 == mach [unitless]
    --- cruise ---
        None
    --- desc1 ---
        [final]   1.0000e+04 == altitude [ft]
    --- desc2 ---
        [final]   1.0000e+03 == altitude [ft]
```

In [ ]:
# Testing Cell
from aviary.api import Mission
from aviary.utils.doctape import glue_variable, get_previous_line, get_variable_name

glue_variable(get_variable_name(Mission.Objectives.FUEL), md_code=True)
glue_variable(get_variable_name(Mission.Objectives.RANGE), md_code=True)

## Beyond level 1

Aviary Level 1 is quite capable yet limited in how much is exposed to the user. If you need additional flexibility or want to add your own subsystems, you will need to move to Levels 2 or 3. Let us briefly discuss some of the possible reasons for you to move to Levels 2 or 3.

If you examine the Aviary metadata, you find that Aviary has two choices of objectives: {glue:md}`Mission.Objectives.FUEL` and {glue:md}`Mission.Objectives.RANGE`. But in level 1, you are limited to minimizing fuel. Users will be able to make other choices in level 2 and level 3.

Level 1 uses the default `phase_info` dictionaries, though there is an option to use a customized `phase_info`.
For more details, please read [drawing and running simple missions](../user_guide/drawing_and_running_simple_missions).

There may be more or fewer key-value pairs in each of the phases. For example, if you have an external subsystem, say, a battery with `BatteryBuilder()`, you can add a key `external_subsystems` with value `BatteryBuilder()`. We will go into details later.

No matter what mission method users choose, they can create their own phase sequences and run Aviary using either level 2 or 3 interface.


We are ready to move on to [Level 2](onboarding_level2).